# 0. Setup

In [ ]:
# pip install --user GitPython
# pip install --user rdkit

In [ ]:
GoogleColab = False

# Verify we're in the correct working directory
import os
from pathlib import Path


if GoogleColab:
  ## mount connection to personal drive
  from google.colab import drive
  drive.mount('/content/drive')

  ## root directory:
  root = "/content/drive/MyDrive/"

  !pip install gitpython
  import git

  root = "/content/drive/MyDrive/dmPC/scripts_model"

else: 
  # Verify we're in the correct working directory

  import git 

  def get_project_root():
      return Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

  root = get_project_root()


os.chdir(root)
os.getcwd()

In [ ]:
if GoogleColab:
  plot_folder = "../images/GDSC/"
else:
  plot_folder = "images/GDSC/"

## import packages, models, trainers

In [ ]:

if GoogleColab:
  !pip install modin
  # pip install -U ray

In [ ]:
# pip install modin

In [ ]:
import argparse
import logging
import sys
import time
import warnings

import numpy as np
# import pandas as pd
import modin.pandas as pd
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200

import torch
from torch import nn, optim, Tensor
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

print('pytorch version:', torch.__version__)
print('orig num threads:', torch.get_num_threads())

In [ ]:
from models import *
from trainers import *
from losses import *
from utils import *
# from cpd_smiles_embed import *

In [ ]:
import random
seed=42

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 1. Prepare dataset

## Load 

In [ ]:
data_folder = "data/GDSC"
c_data = pd.read_csv(os.path.join(data_folder, "c_data.csv"), index_col = 0)
c_meta = pd.read_csv(os.path.join(data_folder, "c_meta.csv"), index_col = 0)
# RNAseq_meta['COSMIC_ID'] = RNAseq_meta['COSMIC_ID'].astype(int)

d_data = pd.read_csv(os.path.join(data_folder, "d_data.csv"), index_col = 0)

cdr = pd.read_csv(os.path.join(data_folder, "cdr.csv"), index_col = 0)

In [ ]:
cdr.shape

## Prepare data
Skin cancer

In [ ]:
c_types = ["ALL","LAML", "LCML", "CLL", "DLBC", "SKCM"]
c_type_str = 'skin_ll'

c_meta = c_meta[c_meta["cancer_type"].isin(c_types)]
c_data = c_data[c_data.index.isin(c_meta["COSMIC_ID"])]
cdr = cdr[cdr.index.isin(c_meta["COSMIC_ID"])]

In [ ]:
cdr.shape

In [ ]:
# 1. prepare c_meta, 
c_meta_id_col_name = 'COSMIC_ID'
c_meta_type_col_name = 'cancer_type'

c_meta = c_meta[[c_meta_id_col_name, c_meta_type_col_name]]
c_meta = c_meta.rename(columns = {c_meta_id_col_name:'C_ID', c_meta_type_col_name:'C_type'})
c_meta = c_meta[~c_meta['C_ID'].isnull()]



In [ ]:
## test stability
test_stability = False
if test_stability:
    # Randomly sample 20% of the DataFrame
    sampled_rows = c_meta.sample(frac=0.15)

    # Set the value of 'C_type' in these rows to 'Other'
    c_meta.loc[sampled_rows.index, 'C_type'] = 'Other'

In [ ]:
new_init_cluster = True
cluster_method = "seld_define" # "KMeans"
if new_init_cluster:
    if cluster_method == "seld_define":
        c_meta.loc[c_meta['C_type'].isin(["ALL","LAML", "LCML", "CLL"]), 'C_type'] = 'leukemia'
        c_meta.loc[c_meta['C_type'].isin(["DLBC"]), 'C_type'] = 'lymphoma'

    if cluster_method == "KMeans":
        import matplotlib.pyplot as plt
        from sklearn.cluster import kmeans_plusplus
        from sklearn.cluster import KMeans
        import seaborn as sns
        import matplotlib.pyplot as plt

        cdr_tmp = cdr.fillna(-1).to_numpy()
        # cdr_tmp.dtypes
        centers_init, indices = kmeans_plusplus(cdr_tmp, n_clusters=2, random_state=0)

        kmeans = KMeans(n_clusters=2, init=centers_init, n_init=1, random_state=0)
        kmeans.fit(cdr_tmp)

        cluster_assignments = kmeans.labels_ 

        combined = np.column_stack((cluster_assignments, cdr_tmp))

        # Sort by cluster assignments
        combined_sorted = combined[combined[:, 0].argsort()]

        # Removing the cluster assignment column for the heatmap
        data_sorted = combined_sorted[:, 1:]

        # Create the heatmap
        plt.figure(figsize=(10, 10))  # You can adjust the size as needed
        sns.heatmap(data_sorted, cmap='viridis')  # Choose a colormap that fits your preferences
        plt.title('Heatmap of CDR Sorted by Cluster Assignments')
        plt.show()

        c_meta.loc['C_type'] = cluster_assignments

In [ ]:
c_meta, meta_map = get_CCL_meta_codes(c_data.index.values, c_meta)
c_meta.index = c_meta.index.astype(str)

print(f"Cancer type coding map: ")
print(meta_map)

In [ ]:
column_counts = c_meta['code'].value_counts()
print(column_counts)

In [ ]:
# 2. prepare c_data
## make sure: 
##   1. the index (row names) is cancer cell line names
c_data.index = c_data.index.astype(str)
c_data.shape

In [ ]:
# 3. prepare d_data
## make sure: 
##   1. the index (row names) is drug names
# cpd_smiles = cpd_smiles[['drug_id', 'smiles']]
# cpd_smiles = cpd_smiles.set_index('drug_id')

# d_data = smiles_to_AtonBondDescriptor_PCAembedings(cpd_smiles)
d_data.index = d_data.index.astype(str)

d_data.shape

In [ ]:
# 4. prepare cdr
## make sure: 
##   1. the index (row names) is cancer cell line names
##   2. the columns (column names) is drug names
cdr.index = cdr.index.astype("str")

common_drugs = list(set(cdr.columns).intersection(set(d_data.index)))
cdr = cdr[common_drugs]
d_data = d_data.loc[common_drugs]

common_cancers = list(set(cdr.index).intersection(set(c_data.index)))
cdr = cdr.loc[common_cancers]
c_data = c_data.loc[common_cancers]
c_meta = c_meta.loc[common_cancers]

print(f'cdr shape: {cdr.shape}')
print(f'c_data shape: {c_data.shape}')
print(f'c_meta shape: {c_meta.shape}')
print(f'd_data shape: {d_data.shape}')

In [ ]:
c_meta.head()

# 2. Hyperparameters

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(device)
print(device)

In [ ]:
class Train_Args:
    def __getitem__(self, key):
        return getattr(self, key)
    def __setitem__(self, key, val):
        setattr(self, key, val)
    def __contains__(self, key):
        return hasattr(self, key)

    valid_size = 0.2 #@param {type: "float"}

    n_epochs = 80 #@param {type: "integer"} # 100
    batch_size = 80 #@param {type: "integer"} # 50
    lr = 0.01 #@param {type: "float"}

    num_workers = 1
    
    C_VAE_loss_weight = 0.5 #@param {type: "float"}
    C_recon_loss_weight = 0.1 #@param {type: "float"}
    C_kld_weight = 0.5 #@param {type: "float"} # 0.5
    C_cluster_distance_weight = 220 #@param {type: "float"}
    C_update_ratio_weight = 10 #@param {type: "float"}"}
    
    D_VAE_loss_weight = 1 #@param {type: "float"}
    D_recon_loss_weight = 1 #@param {type: "float"}
    D_kld_weight = 0.5 #@param {type: "float"}
    D_cluster_distance_weight = 200 #@param {type: "float"}
    D_update_ratio_weight = 10 #@param {type: "float"}
    
    predict_loss_weight = 4000 #@param {type: "float"}
    
    cVAE_save_path = 'data/model_fits/GDSC_' + c_type_str + '_c_vae' #@param
    dVAE_save_path = 'data/model_fits/GDSC_' + c_type_str + '_d_vae' #@param
    
    c_p_save_path = 'data/model_fits/GDSC_' + c_type_str + '_c_vae_predictor' #@param
    d_p_save_path = 'data/model_fits/GDSC_' + c_type_str + '_d_vae_predictor' #@param

    use_weighted_bce = True
    use_mixture_kld = False



class CDPModel_sub_Args:
    def __getitem__(self, key):
        return getattr(self, key)
    def __setitem__(self, key, val):
        setattr(self, key, val)
    def __contains__(self, key):
        return hasattr(self, key)

    # c_VAE
    c_input_dim = 0 #@param {type: "integer"}
    c_h_dims = [1024, 512, 256, 128] #@param {type: "vactor"}
    c_latent_dim = 32 #@param {type: "integer"}

    # d_VAE
    d_input_dim = 0 #@param {type: "integer"}
    d_h_dims = [64]  #@param {type: "vactor"}
    d_latent_dim = 32 #@param {type: "integer"}

    # predictor
    p_sec_dim = 16 #@param {type: "integer"}
    p_h_dims = [p_sec_dim*2, 16]  #@param {type: "vactor"}
    
    # all
    drop_out = 0  #@param {type: "float"}
    
    # sensitive threshold
    sens_cutoff = 0.5
    


In [ ]:
train_args = Train_Args()

K = len(c_meta[c_meta['code'] != -1]['code'].unique())

CDPmodel_args = CDPModel_sub_Args()
CDPmodel_args['c_input_dim'] = c_data.shape[1] 
CDPmodel_args['d_input_dim'] = d_data.shape[1]

if CDPmodel_args['c_input_dim'] <= 0:
  warnings.warn(
      '''\nCancer Cell line feature number not specified''')
if CDPmodel_args['d_input_dim'] <= 0:
  warnings.warn(
      '''\nDrug feature number not specified''')

# 3. Train Model

In [ ]:
CDPmodel = CDPmodel(K, CDPmodel_args)

In [26]:
start_time = time.time()

n_rounds = 4
fit_returns = CDPmodel.fit(c_data, c_meta, d_data, cdr, train_args, n_rounds=n_rounds, search_subcluster=True, device = device)

end_time = time.time()
duration = end_time - start_time
print("Took {} seconds to train".format(duration))

            Best epoc with test loss: epoch 20
            Running time: 3735.4372770786285


KeyboardInterrupt: 

In [ ]:
c_meta, c_meta_hist, d_sens_hist, losses_train_hist_list, best_epos_list, C_VAE_init_losses, D_VAE_init_losses, c_latent_list, d_latent_list = fit_returns

# 4. Results and visualizations

In [ ]:
cdr_hat = CDPmodel.predict(c_data, d_data)

In [ ]:
cdr_train_hat = cdr_hat

cdr_train_rslt = cdr.copy()
cdr_train_rslt['c_name'] = cdr_train_rslt.index.values
cdr_train_rslt = pd.melt(cdr_train_rslt, id_vars='c_name', value_vars=None, var_name=None, value_name='value', col_level=None)
cdr_train_rslt = cdr_train_rslt.rename(columns={'variable':'d_name', 'value':'cdr'})

cdr_train_rslt = pd.merge(cdr_train_rslt, cdr_train_hat, on=['c_name', 'd_name'], how='outer')

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# Binary cross entropy
cdr_train_rslt_noNA = cdr_train_rslt.dropna(subset=['cdr_hat', 'cdr'])
binary_cross_entropy_train = log_loss(cdr_train_rslt_noNA['cdr'], cdr_train_rslt_noNA['cdr_hat'])
print(f"Binary cross entropy: {round(binary_cross_entropy_train,4)}")


# Area Under the Curve (AUC) for a Receiver Operating Characteristic (ROC) 
roc_auc = roc_auc_score(cdr_train_rslt_noNA['cdr'], cdr_train_rslt_noNA['cdr_hat'])
print("ROC AUC:", round(roc_auc, 4))

# confusion_ atrix
cdr_train_rslt_noNA['cdr_hat_bnr'] = (cdr_train_rslt_noNA['cdr_hat'] > 0.5).astype(int)

conf_matrix = confusion_matrix(cdr_train_rslt_noNA['cdr'], cdr_train_rslt_noNA['cdr_hat_bnr'])
tn, fp, fn, tp = conf_matrix.ravel()

print(f"\nTrue Positive:  {tp} ({(tp / (tp + fn)) * 100:.2f} %)")
print(f"False Negative: {fn} ({(fn / (fn + tp)) * 100:.2f} %)")

print(f"True Negative:  {tn} ({(tn / (tn + fp)) * 100:.2f} %)")
print(f"False Positive: {fp} ({(fp / (fp + tn)) * 100:.2f} %)")

In [ ]:
# output data
cdr_hat.to_csv(os.path.join(data_folder, c_type_str + "/GDSC_" + c_type_str + "_cdr_hat.csv"), index=True)

for k_itr in CDPmodel.which_non_empty_cluster:
    c_latent_k = pd.DataFrame(c_latent_list[k_itr][n_rounds-1])
    c_latent_k.index = c_data.index
    c_latent_k.to_csv(os.path.join(data_folder, c_type_str + "/GDSC_" + c_type_str + f"_c_latent_cluster{k_itr}.csv"), index=True)

    d_latent_k = pd.DataFrame(d_latent_list[k_itr][n_rounds-1])
    d_latent_k.index = d_data.index
    d_latent_k.to_csv(os.path.join(data_folder, c_type_str + "/GDSC_" + c_type_str + f"_d_latent_cluster{k_itr}.csv"), index=True)


In [ ]:
cdr_train_hat = CDPmodel.predict(c_data, d_data)

cdr_train_rslt = cdr.copy()
cdr_train_rslt['c_name'] = cdr_train_rslt.index.values
cdr_train_rslt = pd.melt(cdr_train_rslt, id_vars='c_name', value_vars=None, var_name=None, value_name='value', col_level=None)
cdr_train_rslt = cdr_train_rslt.rename(columns={'variable':'d_name', 'value':'cdr'})


cdr_train_rslt = pd.merge(cdr_train_rslt, cdr_train_hat, on=['c_name', 'd_name'], how='outer')

cdr_train_rslt

## Clusters

In [ ]:
print('Cancer clustering before:')
print(c_meta_hist.code.value_counts())
print('Cancer clustering after:')
print(c_meta_hist.code_latest.value_counts())

In [ ]:
c_meta_train_tmp = c_meta.loc[:, ['code']]
c_meta_train_tmp['c_name'] = c_meta_train_tmp.index.values.astype(str)
c_meta_train_tmp = c_meta_train_tmp.rename(columns={'code':'cluster_init'})

cdr_train_rslt_tmp = cdr_train_rslt[['c_name', 'cluster']]
cdr_train_rslt_tmp = cdr_train_rslt_tmp.drop_duplicates()
cdr_train_rslt_tmp['c_name'] = cdr_train_rslt_tmp['c_name'].astype(str)

cdr_train_rslt_cluster = pd.merge(cdr_train_rslt_tmp, c_meta_train_tmp, on='c_name', how='left')

print("CD-bicluster:")
print(pd.crosstab(cdr_train_rslt_cluster['cluster_init'], cdr_train_rslt_cluster['cluster']))



In [ ]:
cdr_train_rslt_tmp = cdr_train_rslt[['c_name', 'c_cluster']]
cdr_train_rslt_tmp = cdr_train_rslt_tmp.drop_duplicates()
cdr_train_rslt_tmp['c_name'] = cdr_train_rslt_tmp['c_name'].astype(str)

cdr_train_rslt_cluster = pd.merge(cdr_train_rslt_tmp, c_meta_train_tmp, on='c_name', how='left')

print("Cancer cluster:")
print(pd.crosstab(cdr_train_rslt_cluster['cluster_init'], cdr_train_rslt_cluster['c_cluster']))


In [ ]:
d_sens_hist

In [ ]:
print('Sensitive to clusters before:')
print(d_sens_hist.sensitive_k.value_counts())
print('Sensitive to clusters after:')
print(d_sens_hist.sensitive_k_latest.value_counts())

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns

from visuals import *

from matplotlib.colors import LinearSegmentedColormap

In [ ]:
cdr_train_hat_wide = cdr_train_hat.pivot(index='c_name', columns='d_name', values='cdr_hat')
cdr_train_hat_wide = cdr_train_hat_wide.apply(pd.to_numeric, errors='coerce')

c_names = [] 
for k in CDPmodel.which_non_empty_cluster:
    c_names_k = CDPmodel.c_name_clusters_in_trainnig[k]
    c_names.extend(CDPmodel.c_name_clusters_in_trainnig[k])

c_names_in_cluster = [c_name for c_name in c_names if c_name in cdr_train_hat_wide.index]
c_names_not_cluster = [c_name for c_name in cdr_train_hat_wide.index if c_name not in c_names]
c_names = c_names_in_cluster
c_names.extend(c_names_not_cluster)


cdr_train_hat_wide = cdr_train_hat_wide.loc[c_names]

d_names = []
for k in CDPmodel.which_non_empty_cluster:
    d_names.extend(CDPmodel.d_name_clusters_in_trainnig[k])
d_names_not_cluster = [col for col in cdr_train_hat_wide.columns if col not in d_names]
d_names.extend(d_names_not_cluster)
cdr_train_hat_wide = cdr_train_hat_wide[d_names]

sns.heatmap(
        cdr_train_hat_wide, 
        annot=False, 
        cmap='PiYG',
        # ax=axs[m],
        xticklabels=True, yticklabels=False)
cmap_custom = LinearSegmentedColormap.from_list(
    'custom', [(0, 'white'), (1, 'green')])

In [ ]:
plots_save_path = 'results/images/GDSC/GDSC_' + c_type_str + '_c_latent'
plot_c_PCA_latent(c_data, n_rounds, fit_returns, model=CDPmodel, plots_save_path=plots_save_path)


In [ ]:
plots_save_path = 'results/images/GDSC/GDSC_' + c_type_str + '_d_latent'
plot_d_PCA_latent(d_data, n_rounds, fit_returns, model=CDPmodel, plots_save_path=plots_save_path)

### Losses:

In [ ]:
for k in range(CDPmodel.K):
    print(f'k = {k}:')
    for b in range(n_rounds):
        print(f'round {b}:')
        plot_training_losses_train_test_2cols(losses_train_hist_list[k][b], best_epoch_1round = best_epos_list[k][b],
                                              plot_save_path=f'results/images/GDSC/GDSC_' + c_type_str + '_losses_k{k}_b{b}.png')
        
        

### 